In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import librosa
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.preprocessing import StandardScaler
import zipfile

In [3]:
zip_path = '/content/drive/MyDrive/Bird sound classification data/archive (1).zip'
metadata_path = '/content/drive/MyDrive/Bird sound classification data/bird_songs_metadata.csv'

metadata = pd.read_csv(metadata_path)

def extract_features_from_spectrogram(file_path, zip_file):
    with zip_file.open(file_path) as file:
        y, sr = librosa.load(file, sr=None)
    stft = librosa.stft(y)
    spectrogram = np.abs(stft)
    mean_spectrogram = np.mean(spectrogram, axis=1)
    std_spectrogram = np.std(spectrogram, axis=1)
    return np.hstack([mean_spectrogram, std_spectrogram])

In [ ]:
features = []
labels = []

with zipfile.ZipFile(zip_path, 'r') as z:
    for index, row in metadata.iterrows():
        file_path = os.path.join('wavfiles', row['filename'])
        if file_path in z.namelist():
            features.append(extract_features_from_spectrogram(file_path, z))
            labels.append(row['species'])
            print()
            print("Extracted features: ",features)

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


Streaming output truncated to the last 5000 lines.
       0.00599172], dtype=float32), array([0.00222616, 0.0019079 , 0.0018627 , ..., 0.00047054, 0.00047419,
       0.0004721 ], dtype=float32), array([0.00178713, 0.00144583, 0.00124293, ..., 0.00056547, 0.00055984,
       0.00055674], dtype=float32), array([0.00184539, 0.00143457, 0.00119098, ..., 0.00034613, 0.00034608,
       0.00034746], dtype=float32), array([0.00570598, 0.00544999, 0.00528402, ..., 0.01714838, 0.01715599,
       0.0171584 ], dtype=float32), array([0.00271274, 0.00249293, 0.00238173, ..., 0.00385138, 0.00384625,
       0.00384052], dtype=float32), array([0.00491876, 0.00489202, 0.00487453, ..., 0.01026108, 0.01025549,
       0.0102639 ], dtype=float32), array([0.00607221, 0.00603026, 0.00595308, ..., 0.01736618, 0.01734201,
       0.01734315], dtype=float32), array([0.0050517 , 0.00506125, 0.00485193, ..., 0.00125856, 0.00130284,
       0.00134548], dtype=float32), array([0.00519747, 0.00492263, 0.00453565, ..., 0

In [ ]:

selector = SelectKBest(score_func=mutual_info_classif, k=10)
X_train_selected = selector.fit_transform(X_train_scaled, y_train)
X_test_selected = selector.transform(X_test_scaled)

selected_features_indices = selector.get_support(indices=True)



print("Indices of selected features:", selected_features_indices)

Indices of selected features: [1209 1227 1228 1229 1230 1231 1232 1233 1244 1264]


In [ ]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_scaled, y_train)
y_pred = knn.predict(X_test_scaled)

In [ ]:
confusion_matrix(y_test, y_pred)

array([[154,   7,  26,   2,   4],
       [  4, 187,   5,   4,   4],
       [ 22,  15, 197,   6,   3],
       [  2,   1,   7, 189,   8],
       [  7,  11,   5,  17, 198]])

In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

    bewickii       0.81      0.80      0.81       193
  cardinalis       0.85      0.92      0.88       204
     melodia       0.82      0.81      0.82       243
 migratorius       0.87      0.91      0.89       207
 polyglottos       0.91      0.83      0.87       238

    accuracy                           0.85      1085
   macro avg       0.85      0.85      0.85      1085
weighted avg       0.85      0.85      0.85      1085

